In [1]:
from pynq import Overlay
from pynq import MMIO
from pynq import allocate
import numpy as np

# Program bitstream to FPGA
overlay = Overlay('/home/xilinx/design_1.bit')

# Access to AXI DMA
dma = overlay.axi_dma_0
dma_send = overlay.axi_dma_0.sendchannel
dma_recv = overlay.axi_dma_0.recvchannel

In [2]:
# Allocate physical memory
input_buffer = allocate(shape=(8,), dtype=np.uint32)
output_buffer = allocate(shape=(8,), dtype=np.uint32)

In [3]:
# Write input
for i in range(8):
    a = i + 1
    b = 8 - i
    y = i + 1
    input_buffer[i] = (y << 16) | (b << 8) | a

In [4]:
# Check the written data
for i in range(8):
    print("0x%08X" % (input_buffer[i]))

0x00010801
0x00020702
0x00030603
0x00040504
0x00050405
0x00060306
0x00070207
0x00080108


In [5]:
# Do AXI DMA MM2S transfer
dma_send.transfer(input_buffer)
# Do AXI DMA S2MM transfer
dma_recv.transfer(output_buffer)

In [6]:
# Check the memory content after DMA transfer
for i in range(8):
    print("%d" % (output_buffer[i]))

9
16
21
24
25
24
21
16


In [7]:
# Delete buffer to prevent memory leak
del input_buffer, output_buffer